In [ ]:
!pip install replicate ipywidgets

In [ ]:
import os
import replicate
from pathlib import Path
from ipywidgets import widgets
from IPython.display import display

In [ ]:
def process_image(image_path, query):
    """
    Process a single image using the Replicate API and return the description.
    """
    output = replicate.run(
        "cjwbw/cogvlm:a5092d718ea77a073e6d8f6969d5c0fb87d0ac7e4cdb7175427331e1798a34ed",
        input={
            "vqa": False,
            "image": open(image_path, "rb"),
            "query": query
        }
    )
    return output

def process_folder(input_folder, output_folder, query):
    """
    Process all images in the input folder and save descriptions to the output folder.
    """
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    
    # Create output folder if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Supported image extensions
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    
    for image_file in input_path.iterdir():
        if image_file.suffix.lower() in image_extensions:
            print(f"Processing {image_file.name}...")
            
            # Generate output file path
            output_file = output_path / f"{image_file.stem}.txt"
            
            # Process image and get description
            description = process_image(str(image_file), query)
            
            # Write description to output file
            with open(output_file, 'w') as f:
                f.write(description)
            
            print(f"Description saved to {output_file}")

# Create input widgets
api_token_input = widgets.Password(
    description='API Token:',
    placeholder='Enter your Replicate API token',
    style={'description_width': 'initial'}
)

query_input = widgets.Text(
    value='Describe this image.',
    placeholder='Enter your query here',
    description='Query:',
    disabled=False,
    style={'description_width': 'initial'}
)

input_folder_input = widgets.Text(
    value='/path/to/your/input/folder',
    placeholder='Enter the input folder path',
    description='Input Folder:',
    disabled=False,
    style={'description_width': 'initial'}
)

output_folder_input = widgets.Text(
    value='/path/to/your/output/folder',
    placeholder='Enter the output folder path',
    description='Output Folder:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Create a button to start processing
process_button = widgets.Button(description="Process Images")

# Display widgets
display(api_token_input, query_input, input_folder_input, output_folder_input, process_button)

# Function to handle button click
def on_button_clicked(b):
    # Set the API token
    os.environ["REPLICATE_API_TOKEN"] = api_token_input.value
    
    # Get values from input widgets
    query = query_input.value
    input_folder = input_folder_input.value
    output_folder = output_folder_input.value
    
    # Process the folder
    process_folder(input_folder, output_folder, query)

# Attach the function to the button
process_button.on_click(on_button_clicked)